In [89]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
from urllib.request import urlopen

## Search words 'vietnam avian flu outbreak'

In [102]:
query = 'vietnam+avian+flu+outbreak'
root = 'https://www.google.com'
first = 'https://www.google.com/search?q='
second = '&biw=1562&bih=885&tbm=nws&sxsrf=ALeKk03WlVR2nPWbPRVygSKAzqjuqBgT3Q%3A1625868930650&ei=gsroYLmeJ8f5-gTZvJqoCQ&oq=vietnam+avian+flu+outbreak&gs_l=psy-ab.3...2587.11721.0.11992.0.0.0.0.0.0.0.0..0.0....0...1c.1.64.psy-ab..0.0.0....0.ZkyOL4xosY8'
link = first + query + second
df = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])

In [91]:
link

'https://www.google.com/search?q=vietnam+avian+flu+outbreak&biw=1562&bih=885&tbm=nws&sxsrf=ALeKk03WlVR2nPWbPRVygSKAzqjuqBgT3Q%3A1625868930650&ei=gsroYLmeJ8f5-gTZvJqoCQ&oq=vietnam+avian+flu+outbreak&gs_l=psy-ab.3...2587.11721.0.11992.0.0.0.0.0.0.0.0..0.0....0...1c.1.64.psy-ab..0.0.0....0.ZkyOL4xosY8'

In [4]:
# def scrape_google_news(link):
#    html = requests.get(link).text
#    soup = BeautifulSoup(html, 'lxml')
#    news = soup.find_all('div', class_ = 'ZINbbc xpd O9g5cc uUPGi')
#    for item in news:
#        title = item.find('div', class_ = 'BNeawe vvjwJb AP7Wnd').text
#        url = item.find('a', href = True)['href'].split('/url?q=')[1]
#        date = item.find('span', class_ = 'r0bn4c rQMQod').text
#        print(title)
#        print(url)
#        print(date)
#        print()
#        document = open('google_news_data.csv', 'a')
#        document.write('{}, {}, {} \n'.format(title, url, date))
#        document.close()
#    next = soup.find('a', class_ = 'nBDE1b G5eFlf')['href']
#    link = root + next
#    scrape_google_news(link)

In [98]:
def scrape_google_news(link, df, start, end):
    if start == end:
        return df
    html = requests.get(link).text
    soup = BeautifulSoup(html, 'lxml')
    news = soup.find_all('div', class_ = 'ZINbbc xpd O9g5cc uUPGi')
    for item in news:
        title = item.find('div', class_ = 'BNeawe vvjwJb AP7Wnd').text
        url = item.find('a', href = True)['href'].split('/url?q=')[1]
        time = item.find('span', class_ = 'r0bn4c rQMQod').text
        d = get_date(time, date.today())
        content = item.find('div', class_ = 'BNeawe s3v9rd AP7Wnd').text.split(' · ')[1]
        new_row = {'Title': title, 'URL': url, 'Date': d, 'Content': content}
        df = df.append(new_row, ignore_index = True)
    next = soup.find('a', class_ = 'nBDE1b G5eFlf')['href']
    link = root + next
    start += 1
    return scrape_google_news(link, df, start, end)

In [99]:
def get_date(time, cur_date):
    x, y = time.split(' ')[0: 2]
    d = None
    if y in ['day', 'days']:
        d = cur_date - timedelta(days = int(x))
    elif y in ['week', 'weeks']:
        d = cur_date - timedelta(weeks = int(x))
    elif y in ['month', 'months']:
        d = cur_date - timedelta(weeks = int(x) * 4)
    elif y in ['year', 'years']:
        d = cur_date - timedelta(weeks = int(x) * 48)
    return d

In [103]:
df = scrape_google_news(link, df, 0, 10)

In [104]:
df

,Title,URL,Date,Content
0,Vietnam confirms first H5N8 avian flu outbreak,https://www.agri-pulse.com/articles/16129-viet...,2021-07-05,Vietnam has confirmed the first outbreak of th...
1,Vietnam reports first H5N8 avian flu outbreak ...,https://www.reuters.com/world/asia-pacific/vie...,2021-07-05,Vietnam reports first H5N8 avian flu outbreak ...
2,An ideal public health model? Vietnam’s state-...,https://www.brookings.edu/blog/order-from-chao...,2021-06-28,While Vietnam is experiencing its largest surg...
3,"News Scan for Jul 06, 2021",https://www.cidrap.umn.edu/news-perspective/20...,2021-07-05,Vietnam recently reported its first highly pat...
4,Ghana culls thousands of chickens amid avian f...,https://arynews.tv/ghana-culls-chickens-avian-...,2021-07-17,"Avian influenza virus occurs mainly in birds, ..."
...,...,...,...,...
95,How to reduce avian flu spread at Asian poultr...,https://www.wattagnet.com/articles/42864-two-s...,2021-05-24,"As recently as one month ago, authorities in V..."
96,Endangered Species Are Paying the Price of COV...,https://foreignpolicy.com/2021/07/11/endangere...,2021-07-12,"Pre-pandemic, a combination of falling prices ..."
97,First known human case of H10N3 influenza in C...,https://www.wattagnet.com/articles/42951-first...,2021-06-21,These were the first new registered cases of a...
98,H5N1 avian influenza: Laos reports first human...,http://outbreaknewstoday.com/h5n1-avian-influe...,2020-12-07,A total of 862 cases of H5N1 influenza have be...


In [109]:
df.to_csv('data/google_news_articles.csv', index = False)

## Search Vietnamese 'dịch cúm gia cầm' (HPAI)

In [105]:
q = 'dịch+cúm+gia+cầm'
link = first + q + second
dt = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])

In [106]:
dt = scrape_google_news(link, dt, 0, 10)

In [107]:
dt

,Title,URL,Date,Content
0,Quảng Ninh phát hiện chủng cúm gia cầm lần đầu...,https://tuoitre.vn/quang-ninh-phat-hien-chung-...,2021-07-05,"Ngay sau đó, Trung tâm Dịch vụ kỹ thuật nông n..."
1,Quảng Ninh xuất hiện cúm gia cầm H5N8,https://vnexpress.net/quang-ninh-xuat-hien-cum...,2021-07-05,Ổ dịch cúm gia cầm được ghi nhận tại một trại ...
2,Lần đầu tiên phát hiện chủng cúm gia cầm A/H5N...,https://vov.vn/xa-hoi/tin-24h/lan-dau-tien-pha...,2021-07-05,Trung tâm Dịch vụ kỹ thuật nông nghiệp thành p...
3,"3 tỉnh có cúm gia cầm A/H5N8 thể độc lực cao, ...",https://thanhnien.vn/tai-chinh-kinh-doanh/3-ti...,2021-07-05,"Theo Bộ NN-PNT cho biết, từ đầu tháng 6 đến na..."
4,"Nghệ An ra công điện khẩn phòng, chống chủng v...",https://baonghean.vn/nghe-an-ra-cong-dien-khan...,2021-07-15,(Baonghean.vn) - Chủng cúm gia cầm A/H5N8 từng...
...,...,...,...,...
95,Nhận diện chủng virus cúm gia cầm độc lực cao ...,https://nongnghiep.vn/nhan-dien-chung-virus-cu...,2021-07-17,"Dịch Covid-19 đang có diễn biến phức tạp, việc..."
96,"Chuyện về ông tiến sĩ ""chế"" văcxin chống cúm g...",https://khoahocdoisong.vn/chuyen-ve-ong-tien-s...,2021-07-12,"(khoahocdoisong.vn) - Năm 2003, dịch cúm gia c..."
97,Các trại gà gần ổ dịch cúm A/H5N8 vẫn an toàn,https://nongnghiep.vn/cac-trai-ga-gan-o-dich-c...,2021-07-05,Các trại gà gần ổ dịch cúm A/H5N8 vẫn an toàn....
98,Bắt vụ vận chuyển gần 12.000 con gia cầm chưa ...,https://baonghean.vn/bat-vu-van-chuyen-gan-120...,2021-07-15,Trung tâm Dịch vụ Nông nghiệp huyện đã tiếp nh...


In [108]:
dt.to_csv('data/google_news_articles_vietnamese.csv', index = False)